### data preprocessing 
##### in this step, a lot of preproccessing will be applied.
##### explanations will be in the cells
##### at the end, number of unique recipes : 40079 
##### number of unique ingredients number: 453 
##### number of unique units: 31

In [ ]:
import pandas as pd
import numpy as np
from initial_functions import *

In [ ]:
ing=pd.read_csv("/Users/osman/Desktop/Master Project/data/1/ingUnitQuant.csv")
nut=pd.read_csv("/Users/osman/Desktop/Master Project/data/1/nutrient.csv")
serv=pd.read_csv("/Users/osman/Desktop/Master Project/data/1/servings.csv")

#truncate the recipes that dont have a serving information
#and convert them to float
#only 3 recipes are truncated

serv=serv[serv["Servings"]!="url error"]
#converting the serving information to int from string
serv['Servings'] = pd.to_numeric(serv['Servings'], errors='coerce').fillna(0).astype(int)
serv=serv[serv["Servings"]!=0].reset_index(drop=True)
serv=serv.drop(serv.columns[0], axis=1)

In [ ]:
#adding servings information to the ingredient table
ing1=ing.merge(serv, on='recipe', how='inner')
#convert everthing lower case
ing1['ingredientName'] = ing1['ingredientName'].str.lower()
ing1['ingredientUnit'] = ing1['ingredientUnit'].str.lower()


#delete units' recipes that are used less than 50 times
ing1=trun(ing1, "URL", "ingredientUnit",50)

In [ ]:
#delete the rows(ingredients) that have NaN values in unit and quantity at the same time

ing1=ing1[~(ing1.ingredientQuantity.isnull() & ing1.ingredientUnit.isnull())].reset_index(drop=True)

In [ ]:
#adding "piece" as a unit if the unit is still NaN
#because almost all of them meant to be "piece" i.e 1 potato, 1 onion etc.
ing1.ingredientUnit.fillna("piece", inplace=True)

In [ ]:
#deleting recipes that have NaN values in quantity is necessary. Especially for salt. We need to know amount of salt.
#because we will use the quantity information to calculate the nutrient values.And salt and others are important.
#deleting only those rows would have been wrong.Bcs it cahanges the recipe.
#but deleting only rows can be tried as well before ml part.
#so in this step we are losing aroung 4k recipes

print(ing1[ing1.ingredientQuantity.isnull()].URL.nunique())
delrec=ing1[ing1.ingredientQuantity.isnull()].URL.unique() #recipes that have NaN values in quantity
ing1=ing1[~ing1.URL.isin(delrec)].reset_index(drop=True) #deleting those recipes

In [ ]:
#recipes that have less than 3 ingredients will be truncated. about 1k recipes
#we loose 1254 recipes here
print(ing1[(ing1['recipe'].map(ing1['recipe'].value_counts()) < 3)].URL.nunique())
ing1=ing1[~(ing1['recipe'].map(ing1['recipe'].value_counts()) < 3)].reset_index(drop=True)

#delete the recipes that have ingredients which are used less then 100 in our dataset
#this one truncates the whole recipes, NOT the ingredients!
#we had 5877 unique igredients before this step. 
#5424 of themwill be truncated after this step. This will cause loosing 30542 recipes (from 70k to 40k) 
# these are dramatic numbers, needed to be considered again and this step might be passed before ml steps for a try.

ing3=trun(ing1, "URL", "ingredientName",100)
ing3.URL.nunique()

In [ ]:
#converting the str values of quantities to float by using an external function

ing3['ingredientQuantity'] = ing3['ingredientQuantity'].apply(convert_to_float)

#10 recipes that still have NaN values in quantity will be truncated
print(ing3[ing3.ingredientQuantity.isnull()].URL.nunique())
delrec=ing3[ing3.ingredientQuantity.isnull()].URL.unique() #recipes that have NaN values in quantity
ing3=ing3[~ing3.URL.isin(delrec)].reset_index(drop=True) #deleting those recipes


In [ ]:
# first part of the preprocessing is done.
#there is no missing values in the dataset now.
#unit normalization might be done before ml part in the next step.
print("recipe number:",ing3.URL.nunique(),"\n",
      "unique ingredient number:",ing3.ingredientName.nunique(),"\n",
      "unique unit number:",ing3.ingredientUnit.nunique())

In [ ]:
#save to csv
ing3.to_csv("/Users/osman/Desktop/Master Project/data/2/recipe40k.csv", index=False)